In [69]:
---
title: "test"
format:
    pdf:
        toc: true
        number-section: true
        colorlinks: true
jupyter: python3
---

SyntaxError: invalid syntax (1533296891.py, line 1)

In [38]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT # <-- ADD THIS LINE

In [ ]:
# run these commands inside
!docker run --name pg_local -p 5432:5432 -e POSTGRES_USER=sde -e POSTGRES_PASSWORD=password -e POSTGRES_DB=scd2 -d postgres:12.2

In [ ]:
!pgcli -h localhost -p 5432 -U sde scd2 
# password is password

In [65]:
# make sure you have docker running
# and your postgresql container running
!docker ps

CONTAINER ID   IMAGE           COMMAND                  CREATED          STATUS          PORTS                    NAMES
e96549e4bce0   postgres:12.2   "docker-entrypoint.s…"   46 minutes ago   Up 46 minutes   0.0.0.0:5432->5432/tcp   pg_local


In [39]:
dbname='scd2'
user = 'sde'
host='localhost:5432'
password = 'password'

In [47]:
connection = psycopg2.connect(f"dbname={dbname} user={user} password={password}")

In [48]:
connection.autocommit = True

In [49]:
cur = connection.cursor()

In [50]:
print('PostgreSQL database version:')
cur.execute('SELECT version()')

PostgreSQL database version:


In [51]:
db_version = cur.fetchone()

In [52]:
print(db_version)

('PostgreSQL 12.2 (Debian 12.2-2.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)


In [53]:
command = """
CREATE DATABASE warehouse;
"""

In [54]:
cur.execute(command)

In [59]:
# create user table and fill it
user_table_command = """
DROP TABLE IF EXISTS user_dim;
CREATE TABLE user_dim (
    user_key BIGINT,
    user_id VARCHAR(40),
    first_name VARCHAR(10),
    last_name VARCHAR(10),
    address VARCHAR(100),
    zipcode VARCHAR(10),
    created_datetime TIMESTAMP,
    updated_datetime TIMESTAMP,
    row_effective_datetime TIMESTAMP,
    row_expiration_datetime TIMESTAMP,
    current_row_indicator VARCHAR(10)
);
INSERT INTO user_dim (
        user_key,
        user_id,
        first_name,
        last_name,
        address,
        zipcode,
        created_datetime,
        updated_datetime,
        row_effective_datetime,
        row_expiration_datetime,
        current_row_indicator
    )
VALUES (
        1000,
        'b0cc9fde-a29a-498e-824f-e52399991beb',
        'john',
        'doe',
        'world',
        10027,
        '2020-01-01 10:00:00',
        '2020-01-01 10:00:00',
        ' 2020-01-01 10:00:00',
        '2021-01-01 17:59:59',
        'expired'
    ),
    (
        1200,
        'b0cc9fde-a29a-498e-824f-e52399991beb',
        'john',
        'doe',
        'world',
        10012,
        '2020-01-01 10:00:00',
        '2021-01-01 18:00:00',
        '2021-01-01 18:00:00',
        '9999-12-31 00:00:00',
        'current'
    );
"""

In [60]:
cur.execute(user_table_command)

In [61]:
# now we check our work
select_user_table = """
select *
from user_dim
"""

In [62]:
cur.execute(select_user_table)

In [63]:
records = cur.fetchall()

In [64]:
for row in records:
    print(row)

(1000, 'b0cc9fde-a29a-498e-824f-e52399991beb', 'john', 'doe', 'world', '10027', datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2021, 1, 1, 17, 59, 59), 'expired')
(1200, 'b0cc9fde-a29a-498e-824f-e52399991beb', 'john', 'doe', 'world', '10012', datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2021, 1, 1, 18, 0), datetime.datetime(2021, 1, 1, 18, 0), datetime.datetime(9999, 12, 31, 0, 0), 'current')
